In [1]:
import json
import numpy as np
import torch
import os 
import random
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torch.utils.data as Data
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    #torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

In [3]:
class MLP(nn.Module):

    def __init__(self, input_dim, embedding_dim, units,output_dim,num_layers=1) -> None:
        super(MLP, self).__init__()
        self.input_dim = input_dim
        self.units = units
        self.output_dim = output_dim
        self.num_layers=num_layers
        self.embedding_dim=embedding_dim
        self.embedding=nn.Embedding(self.input_dim,self.embedding_dim)
        self.embedding(torch.autograd.Variable(torch.LongTensor([0,1,2])))
        self.rnn=nn.GRU(self.input_dim+self.embedding_dim+1,self.units,batch_first=True,num_layers=self.num_layers)
        self.linear1 = nn.Linear(in_features=self.units,
                              out_features=self.units,
                              bias=True)
        self.linear2 =  nn.Linear(in_features=self.units,
                              out_features=self.output_dim,
                              bias=True)
        self.dropout = nn.Dropout(0.2)

    def forward(self, input) -> torch.Tensor:
        x=input[:,:,:51]
        y=input[:,:,51]
        y=self.embedding(y.long())
        x=torch.cat([x,y],dim=2)
        x,_=self.rnn(x)
        x=self.dropout(x[:,-1,:])
        x=F.relu(self.linear1(x))
        output=self.linear2(x)
        return output

In [4]:
def train(model,loader,test_loader,optimizer,loss_function,epoch,patience):
    min_loss=float('inf')
    count=0
    for n in range(epoch):
        model.train()
        loss_all=0.0
        for i,data in enumerate(loader):
            optimizer.zero_grad()
            inputs,labels=data
            inputs=inputs.to(device)
            labels=labels.to(device)
            out=model(inputs) 
            loss=loss_function(out,labels)
            loss_all+=loss
            loss.backward()
            optimizer.step()
        #print(n,loss_all)
        model.eval()
        loss_all=0.0
        for i,data in enumerate(test_loader):
            optimizer.zero_grad()
            inputs,labels=data
            inputs=inputs.to(device)
            labels=labels.to(device)
            with torch.no_grad():
                out=model(inputs) 
            loss=loss_function(out,labels)
            loss_all+=loss
        if loss_all.item()<min_loss:
            count=0
            best_dict=model.state_dict()
            min_loss=loss_all.item()
        else:
            count+=1
        if count>=patience:
            model.load_state_dict(best_dict)
            break

In [5]:
def test(model,loader,loss_function):
    model.eval()
    loss_all=0.0
    for i,data in enumerate(loader):
        optimizer.zero_grad()
        inputs,labels=data
        inputs=inputs.to(device)
        labels=labels.to(device)
        with torch.no_grad():
            out=model(inputs) 
        loss=loss_function(out,labels)
        loss_all+=loss
    print(loss_all)

In [6]:
topic=[[30,18,23,27,2,25,43,37,7,17],[16,16,17,5,5,22,0,25,14,14],[47,0,32,33,25,44,2,26,35,48],[8,2,41,35,1,43,28,19,41,11],[28,3,30,44,34,41,11,34,41,21]]

In [ ]:
seeds=[1029,1031,1032]

In [7]:
p=np.zeros([5,10])

In [8]:
for seed in seeds:
    predictions=[]
    seed_torch(seed)
    for n in range(5):
        data=pd.read_csv('Task_1_data_private/Task_1_data_private_csv/dataset_'+str(n)+'/train.csv',header=None).to_numpy()
        Y_d=data[:,1].reshape(100,400,1)
        Z_d=np.zeros([100,400,1])
        for i in range(100):
            for j in range(400):
                Z_d[i][j][0]=data[400*i+j,2+int(Y_d[i][j][0])]
        data_mean=np.mean(data[:,2:],axis=0)
        data_std=np.std(data[:,2:],axis=0)
        data=(data[:,2:]-data_mean)/data_std
        X=data.reshape(100,400,50)
        X_data=np.concatenate((X,Z_d,Y_d),axis=2)

        X_all=[]
        y_all=[]
        for i in range(100):
            for j in range(300):
                X_all.append(X_data[i,j:j+100])
                y_all.append(X_data[i,j+100,:50])

        X_all=np.array(X_all)
        y_all=np.array(y_all)

        X_all=torch.FloatTensor(X_all)
        y_all=torch.FloatTensor(y_all)

        X_train, X_test, y_train,y_test = train_test_split(X_all, y_all, test_size=0.2,random_state=seed)

        train_dataset=Data.TensorDataset(X_train, y_train)
        test_dataset=Data.TensorDataset(X_test, y_test)

        train_loader = Data.DataLoader(dataset=train_dataset,batch_size=64,shuffle=True,num_workers=0,worker_init_fn=seed)
        test_loader = Data.DataLoader(dataset=test_dataset,batch_size=64,shuffle=False,num_workers=0,worker_init_fn=seed)

        device='cuda'
        model=MLP(50,200,256,50).to(device)
        optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
        loss_function=nn.MSELoss()

        train(model,train_loader,test_loader,optimizer,loss_function,200,10)
        test(model,test_loader,loss_function)
        test_data=json.load(open('Task_2_data_private/Task_2_data_private/intervention_'+str(n)+'.json','r',encoding="utf-8"))

        model.eval()
        pred=[]
        for i in range(10):
            X=np.array(test_data[i]['conditioning'])
            Y_d=X[:,0].reshape(len(X),1)
            Z_d=np.zeros([len(X),1])
            for j in range(len(X)):
                Z_d[j][0]=X[j,1+int(Y_d[j][0])]
            z1=test_data[i]['intervention'][0][0]
            z2=test_data[i]['reference'][0][0]
            X=(X[:,1:]-data_mean)/data_std

            X_data=np.concatenate((X,Z_d,Y_d),axis=1)
            X_data=torch.FloatTensor(X_data).to(device)
            y_1=model(X_data.unsqueeze(0))
            X_data1=torch.cat([X_data,torch.cat([y_1[0],(y_1[0][int(z1)]*data_std[int(z1)]+data_mean[int(z1)]).unsqueeze(0),torch.FloatTensor([z1]).to(device)]).unsqueeze(0)],dim=0)
            X_data2=torch.cat([X_data,torch.cat([y_1[0],(y_1[0][int(z2)]*data_std[int(z2)]+data_mean[int(z2)]).unsqueeze(0),torch.FloatTensor([z2]).to(device)]).unsqueeze(0)],dim=0)

            y_2=model(X_data1.unsqueeze(0))
            y_3=model(X_data2.unsqueeze(0))
            pred.append((y_2[0][topic[n][i]].item()-y_3[0][topic[n][i]].item())*data_std[topic[n][i]])
        predictions.append(pred)
    predictions=np.array(predictions)
    p=p+predictions

0 tensor(30.0349, device='cuda:0', grad_fn=<AddBackward0>)
1 tensor(7.7794, device='cuda:0', grad_fn=<AddBackward0>)
2 tensor(5.3100, device='cuda:0', grad_fn=<AddBackward0>)
3 tensor(4.4225, device='cuda:0', grad_fn=<AddBackward0>)
4 tensor(3.9349, device='cuda:0', grad_fn=<AddBackward0>)
5 tensor(3.5372, device='cuda:0', grad_fn=<AddBackward0>)
6 tensor(3.3786, device='cuda:0', grad_fn=<AddBackward0>)
7 tensor(3.2042, device='cuda:0', grad_fn=<AddBackward0>)
8 tensor(3.0174, device='cuda:0', grad_fn=<AddBackward0>)
9 tensor(2.8548, device='cuda:0', grad_fn=<AddBackward0>)
10 tensor(2.7614, device='cuda:0', grad_fn=<AddBackward0>)
11 tensor(2.6755, device='cuda:0', grad_fn=<AddBackward0>)
12 tensor(2.5805, device='cuda:0', grad_fn=<AddBackward0>)
13 tensor(2.4673, device='cuda:0', grad_fn=<AddBackward0>)
14 tensor(2.3574, device='cuda:0', grad_fn=<AddBackward0>)
15 tensor(2.2628, device='cuda:0', grad_fn=<AddBackward0>)
16 tensor(2.2185, device='cuda:0', grad_fn=<AddBackward0>)
17 ten

In [ ]:
np.save('Task_2_data_private/Task_2_data_private/cate_estimate.npy',p/3)